In [25]:
from keras.models import Model
from keras import layers
from keras.layers import Dense, Input, BatchNormalization, Activation , Flatten ,Reshape
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D,Softmax,Lambda
# from keras.applications.imagenet_utils import _obtain_input_shape
# from keras.utils.data_utils import get_file
import tensorflow as tf

def vol_GAP(x):
    return tf.keras.backend.mean(x,axis=(1,2),keepdims=True)

In [28]:
def Xception():
    # Determine proper input shape
    # input_shape = _obtain_input_shape(None, default_size=299, min_size=71, data_format='channels_last', include_top=False)

    img_input = Input(shape=(64,64,3))
    
    # Block 1
    x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False)(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    residual = Conv2D(128, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    # Block 2
    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    # Block 2 Pool
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    residual = Conv2D(256, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    # Block 3
    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    # Block 3 Pool
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    residual = Conv2D(728, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    # Block 4
    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # Block 5 - 12
    for i in range(8):
        residual = x

        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)

        x = layers.add([x, residual])

    residual = Conv2D(1024, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
    residual = BatchNormalization()(residual)

    # Block 13
    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(1024, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)

    # Block 13 Pool
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # Block 14
    x = SeparableConv2D(1536, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Block 14 part 2
    x = SeparableConv2D(2048, (3, 3), padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

#     x = Conv2D(1024, (1, 1), padding='same', use_bias=False)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)

#     x = Conv2D(512, (1, 1), padding='same', use_bias=False)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
    
#     x = Conv2D(200, (1, 1), padding='same', use_bias=False)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)

    x = GlobalAveragePooling2D()(x)
    x = Reshape((1,1,2048))(x)
    x = Conv2D(200,(1,1),activation='softmax')(x)
    x = Flatten()(x)
    inputs = img_input

    # Create model
    model = Model(inputs, x, name='xception')
    model.summary()
    return model


In [29]:
xcept = Xception()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 31, 31, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_162 (BatchN (None, 31, 31, 32)   128         conv2d_28[0][0]                  
__________________________________________________________________________________________________
activation_142 (Activation)     (None, 31, 31, 32)   0           batch_normalization_162[0][0]    
__________________________________________________________________________________________________
conv2d_29 

In [30]:
import pickle 

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [31]:
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.layers import Dropout
from keras.layers.merge import concatenate
from keras.layers import BatchNormalization
from keras.layers import activations
from keras.layers.core import Dense,Dropout,Activation,Flatten,Lambda
import numpy as np

In [32]:
train_x = load_obj("rgb_only")[:60000]
train_y = load_obj("train_labels")[:60000]

In [33]:
len(train_x)

60000

In [35]:
xcept.compile(optimizer = Adam(lr = 1e-4), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
xcept.fit(np.array(train_x),np.array(train_y), epochs=10, batch_size=10)

TypeError: __int__ returned non-int (type NoneType)